# 전체 데이터셋을 임의로 250 개의 Training dataset 과 71 개의 Validation dataset 으로 구분한 뒤 다음 각 물음에 답하시오. 

In [1]:
install.packages("glmnet")
install.packages("GA")
library(glmnet)
library(GA)

package 'glmnet' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lmwlm\AppData\Local\Temp\RtmpWM2rRj\downloaded_packages
package 'GA' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lmwlm\AppData\Local\Temp\RtmpWM2rRj\downloaded_packages


Loading required package: Matrix

Loaded glmnet 3.0-2


Loading required package: foreach

Loading required package: iterators

Package 'GA' version 3.2
Type 'citation("GA")' for citing this R package in publications.


Attaching package: 'GA'


The following object is masked from 'package:utils':

    de




In [2]:
df <- read.csv("Weather_Ankara.csv")
df_x <- df[,-10]
df_x_scaled <- scale(df_x, center = TRUE, scale = TRUE) #scaling실행
Mean_temperature <- df$Mean_temperature
df_data_scaled <- data.frame(df_x_scaled, Mean_temperature)

In [3]:
nR <- nrow(df_data_scaled)
nC <- ncol(df_data_scaled)

In [4]:
set.seed(41) # 시드값 설정
train_idx <- sample(1:nR, 250) 
train <- df_data_scaled[train_idx,] #250개 train set
val <- df_data_scaled[-train_idx,] #나머지는 val set

# [Q1] 모든 변수를 사용하여 Multiple Linear Regression (MLR) 모형을 학습하시오. 학습된 모형의 Adjusted R2 는 얼마인가? 또한, 유의수준 1% (significance level = 0.01)에서 통계적으로 유의미한 변수는 어떤 것이 있는가? 학습한 모형을 이용하여 Validation dataset 에 대한 RMSE, MAE, MAPE 를 산출해 보시오. 

In [5]:
# RMSE, MAE, MAPE 계산 함수 
perf_eval_reg <- function(label, pred){ #label 정닶, pred 예측값
    
    #RMSE 계산
    rmse <- sqrt(mean(label - pred)^2)
    
    #MAE 계산
    mae <- mean(abs(label - pred))
    
    #MAPE 계산
    mape <- 100*mean(abs((label - pred) / label))
    
    return(c(rmse, mae, mape))
}

perf_mat <- matrix(0, nrow=6, ncol=6)
rownames(perf_mat) <- c("full model","Exhaustive Search","Forward","Backward","Stepwise","Genetic")
colnames(perf_mat) <- c("input_formula","Adj.R^2","Time","RMSE", "MAE", "MAPE")

In [6]:
#MLR
start <- proc.time()
full_model <- lm(train$Mean_temperature ~ ., data = train)
end <- proc.time()
summary(full_model)


Call:
lm(formula = train$Mean_temperature ~ ., data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1461 -1.0427 -0.0844  0.9258  5.1263 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        48.88077    0.10004 488.627   <2e-16 ***
Max_termperature    9.58051    0.27746  34.530   <2e-16 ***
Min_temperature     4.67759    0.33349  14.026   <2e-16 ***
Dewpoint            0.37762    0.27569   1.370   0.1720    
Precipitation       0.01274    0.10566   0.121   0.9041    
Sea_level_pressure -1.27069    0.73345  -1.732   0.0845 .  
Standard_pressure   0.68937    0.57703   1.195   0.2334    
Visibility          0.28922    0.13164   2.197   0.0290 *  
Wind_speed          0.24654    0.15863   1.554   0.1215    
Max_wind_speed     -0.07222    0.14716  -0.491   0.6240    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.575 on 240 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squa

In [7]:
perf_mat[1,1] <- paste(colnames(train[1:9,]),collapse = "+") # 모델에 들어간 변수들 넣기
perf_mat[1,2] <- summary(full_model)$adj.r.squared # Adjusted R-squared  값 저장
perf_mat[1,3] <- (end - start)[3] # 시간 넣기

In [8]:
full_model_pred <- predict(full_model, newdata = val) # validation 셋으로 예측
perf_mat[1,4:6] <- perf_eval_reg(val$Mean_temperature, full_model_pred) # 예측 후 평가지표 계산하여 넣기
perf_mat

,input_formula,Adj.R^2,Time,RMSE,MAE,MAPE
full model,Max_termperature+Min_temperature+Dewpoint+Precipitation+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed+Max_wind_speed+Mean_temperature,0.989044028361043,0,0.184118990750289,1.28235010615204,2.83162660862481
Exhaustive Search,0,0,0,0,0,0
Forward,0,0,0,0,0,0
Backward,0,0,0,0,0,0
Stepwise,0,0,0,0,0,0
Genetic,0,0,0,0,0,0


# [Q2] Exhaustive Search를 수행하는 함수를 직접 구현하고 Training Dataset에 대한 Adjusted R2 기준으 로 가장 높은 값이 산출된 변수 집합을 제시하시오. 또한 Exhaustive Search에 소요된 시간을 산출하시오. 학습한 모형을 이용하여 Validation dataset에 대한 RMSE, MAE, MAPE를 산출하고 모든 변수를 사용한 MLR 모형의 결과와 비교하시오. 

In [9]:
# 비트 연산자 구현
# 숫자 n을 받아 1 ~ 2^n 까지 2진수로 만들어 매트릭스 형태로 리턴
beat_function <- function(n){
    mat <- matrix(0, nrow=2^n-1, ncol=n)
    for(i in 1:(2^n-1)){
        vec <- c()
        tmp <- i
        
        # 2진수로 변환하는 부분
        for(j in 1:n){  
            vec <- append(i%%2,vec)
            i <- i%/%2
        }
        mat[tmp,1:n] <- vec
    }
    return(mat)
}

In [10]:
# exhaustive_search 구현
exhaustive_search <- function(n, train){
    beat_mat <- beat_function(n) # 비트로 만드는 함수로 매트릭스 구함
    best_i <- 0 # 가장 좋은 경우 저장할 곳
    best_adj_R <- 0 # 가장 좋은 경우의 adj.R 저장할 곳
    cols <- c(1:n) # 칼럼들의 숫자 index
    for (i in 1:(2^n-1)){
        # cols와 beat로 된 매트릭스의 행을 차레로 곱해서 모든 경우의 수를 구함
        model <- lm(train$Mean_temperature ~ ., data = train[cols * beat_mat[i,]]) 
        adj_R <- round(summary(model)$adj.r.squared,digits = 5)
        print(paste("adj.R^2:",adj_R,",formular: ",(paste(colnames(train[cols *beat_mat[i,]]),collapse="+"))))
        
        # best가 바꼈을 경우, best_i와 best_adj_R을 바꿔준다.
        if (adj_R > best_adj_R){
            best_i <- i
            best_adj_R <- adj_R
        }
    }
    return(cols * beat_mat[best_i,]) # 최적의 조합을 리턴
}

In [11]:
start <- proc.time()
best_cols <- exhaustive_search(9,train)
end <- proc.time()
perf_mat[2,3] <- (end - start)[3]

[1] "adj.R^2: 0.06704 ,formular:  Max_wind_speed"
[1] "adj.R^2: 0.03433 ,formular:  Wind_speed"
[1] "adj.R^2: 0.06365 ,formular:  Wind_speed+Max_wind_speed"
[1] "adj.R^2: 0.26554 ,formular:  Visibility"
[1] "adj.R^2: 0.2734 ,formular:  Visibility+Max_wind_speed"
[1] "adj.R^2: 0.26261 ,formular:  Visibility+Wind_speed"
[1] "adj.R^2: 0.27949 ,formular:  Visibility+Wind_speed+Max_wind_speed"
[1] "adj.R^2: 0.08203 ,formular:  Standard_pressure"
[1] "adj.R^2: 0.11085 ,formular:  Standard_pressure+Max_wind_speed"
[1] "adj.R^2: 0.08656 ,formular:  Standard_pressure+Wind_speed"
[1] "adj.R^2: 0.10819 ,formular:  Standard_pressure+Wind_speed+Max_wind_speed"
[1] "adj.R^2: 0.29704 ,formular:  Standard_pressure+Visibility"
[1] "adj.R^2: 0.29713 ,formular:  Standard_pressure+Visibility+Max_wind_speed"
[1] "adj.R^2: 0.29948 ,formular:  Standard_pressure+Visibility+Wind_speed"
[1] "adj.R^2: 0.31165 ,formular:  Standard_pressure+Visibility+Wind_speed+Max_wind_speed"
[1] "adj.R^2: 0.40954 ,formular:  Se

In [12]:
es_model <- lm(train$Mean_temperature ~ ., data = train[best_cols]) # best_cols에 저장해둔 조합만 가지고 모델 생성
summary(es_model)
perf_mat[2,2] <- summary(es_model)$adj.r.squared
perf_mat[2,1] <- paste(colnames(train[best_cols]),collapse = "+")
es_model_pred <- predict(es_model, newdata = val)
perf_mat[2,4:6] <- perf_eval_reg(val$Mean_temperature, es_model_pred)
perf_mat


Call:
lm(formula = train$Mean_temperature ~ ., data = train[best_cols])

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1462 -1.0348 -0.0791  0.9466  5.1278 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        48.88231    0.09963 490.656   <2e-16 ***
Max_termperature    9.55883    0.26825  35.634   <2e-16 ***
Min_temperature     4.70258    0.32695  14.383   <2e-16 ***
Dewpoint            0.36353    0.26767   1.358   0.1757    
Sea_level_pressure -1.27200    0.72929  -1.744   0.0824 .  
Standard_pressure   0.69267    0.57471   1.205   0.2293    
Visibility          0.28994    0.13069   2.219   0.0274 *  
Wind_speed          0.19787    0.12422   1.593   0.1125    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.569 on 242 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squared:  0.9891 
F-statistic:  3236 on 7 and 242 DF,  p-value: < 2.2e-16


,input_formula,Adj.R^2,Time,RMSE,MAE,MAPE
full model,Max_termperature+Min_temperature+Dewpoint+Precipitation+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed+Max_wind_speed+Mean_temperature,0.989044028361043,0,0.184118990750289,1.28235010615204,2.83162660862481
Exhaustive Search,Max_termperature+Min_temperature+Dewpoint+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed,0.989123124838608,0.650000000000002,0.177143175569796,1.2808548015898,2.82939076638354
Forward,0,0,0,0,0,0
Backward,0,0,0,0,0,0
Stepwise,0,0,0,0,0,0
Genetic,0,0,0,0,0,0


# [Q3] Forward Selection, Backward Elimination, Stepwise Selection 방식을 사용하여 MLR변수 선택 과정을 수행해 보시오. 각 방법론마다 [Q2]와 같이 Training dataset에 대한 Adjusted R2 및 소요 시간, Validation dataset에 대한 RMSE, MAE, MAPE를  산출하고 [Q1] 및 [Q2]의 결과와 비교하시오. 

In [13]:
start <- c("train$Mean_temperature ~ 1", "full_model", "full_model") #포문을 돌리기 위해 시작모델 저장
method <- c("forward", "backward", "both") # 포문을 돌리기 위해 메소드 저장

In [14]:
# 포문을 사용하여 3가지 방식을 돌렸습니다.
for(i in 1:3){
    print(method[i])
    start_time <- proc.time()
    model <- step(lm(as.formula(start[i]), data =train), 
                    scope = list(upper = full_model, lower = train$Mean_temperature ~ 1), 
                    direction= method[i], trace = 1)
    end_time <- proc.time()
    perf_mat[i+2,3] <- (end_time - start_time)[3]
    print("---------------")
    print(summary(model))
    perf_mat[i+2,2] <- summary(model)$adj.r.squared
    perf_mat[i+2,1] <- as.character(summary(model)$call)[2]
    pred <- predict(model, newdata = val)
    perf_mat[i+2,4:6] <- perf_eval_reg(val$Mean_temperature, pred)
}

[1] "forward"
Start:  AIC=1356.59
train$Mean_temperature ~ 1

                     Df Sum of Sq   RSS     AIC
+ Max_termperature    1     53417  2960  621.87
+ Min_temperature     1     49449  6928  834.47
+ Dewpoint            1     43575 12802  987.98
+ Sea_level_pressure  1     23223 33155 1225.87
+ Visibility          1     15137 41241 1280.43
+ Standard_pressure   1      4833 51545 1336.19
+ Max_wind_speed      1      3990 52387 1340.24
+ Wind_speed          1      2154 54223 1348.85
<none>                            56377 1356.59
+ Precipitation       1       143 56234 1357.96

Step:  AIC=621.87
train$Mean_temperature ~ Max_termperature

                     Df Sum of Sq     RSS    AIC
+ Min_temperature     1   2291.86  668.16 251.77
+ Dewpoint            1   1291.59 1668.43 480.54
+ Sea_level_pressure  1    972.69 1987.33 524.27
+ Standard_pressure   1    695.51 2264.51 556.91
+ Wind_speed          1    665.99 2294.03 560.15
+ Max_wind_speed      1    257.92 2702.10 601.08
+ Vis

In [15]:
perf_mat

,input_formula,Adj.R^2,Time,RMSE,MAE,MAPE
full model,Max_termperature+Min_temperature+Dewpoint+Precipitation+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed+Max_wind_speed+Mean_temperature,0.989044028361043,0,0.184118990750289,1.28235010615204,2.83162660862481
Exhaustive Search,Max_termperature+Min_temperature+Dewpoint+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed,0.989123124838608,0.650000000000002,0.177143175569796,1.2808548015898,2.82939076638354
Forward,train$Mean_temperature ~ Max_termperature + Min_temperature + Sea_level_pressure + Visibility + Standard_pressure,0.989049139340133,0.0300000000000011,0.16406155216688,1.31114759735903,2.89613751259997
Backward,train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Visibility + Wind_speed,0.989102865435691,0.0299999999999976,0.177493145133962,1.29580955253699,2.85758992183211
Stepwise,train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Visibility + Wind_speed,0.989102865435691,0.0300000000000011,0.177493145133962,1.29580955253699,2.85758992183211
Genetic,0,0,0,0,0,0


# [Q4] Adjusted R2를 Fitness function으로 하는 Genetic Algorithm 기반의 변수 선택 함수를 작성해 보시오. 작성한 함수를 이용하여 GA를 이용한 변수 선택을 수행한 결과를 소요 시간, Adjusted R2, Validation dataset에 대한 RMSE, MAE, MAPE 관점에서 앞선 결과들과 비교해 보시오.

In [16]:
# GN에서 fitnees functiond으로 사용될 함수 작성
# lm모델을 만든 후에, adj.r을 가져와 리턴해 준다.
fitness <- function(string){
  sel_var_idx <- which(string == 1) 
  sel_x_train <- x_train[, sel_var_idx] 
  tmp <- data.frame(sel_x_train, y_train)
  model <- lm(y_train ~ ., data = tmp)
  return(summary(model)$adj.r.squared)
}

In [17]:
x_train <- as.matrix(train[,-10])
y_train <- train[,10]
# 파라미터들은 실습 코드의 파리미터를 썼습니다. 문제 5번에서 바꿔보겠습니다.
start_time <- proc.time()
GN <- ga(type = "binary", fitness = fitness, nBits = ncol(x_train), 
            names = colnames(x_train), popSize = 50, pcrossover = 0.5, 
            pmutation = 0.01, maxiter = 100, elitism = 2, seed = 41)
end_time <- proc.time()
perf_mat[6,3] <- (end_time - start_time)[3]
best_var_idx <- which(GN@solution == 1)
colnames(train)[best_var_idx]

[1] "Max_termperature"   "Min_temperature"    "Dewpoint"          
[4] "Sea_level_pressure" "Standard_pressure"  "Visibility"        
[7] "Wind_speed"

In [18]:
summary(GN)

-- Genetic Algorithm ------------------- 

GA settings: 
Type                  =  binary 
Population size       =  50 
Number of generations =  100 
Elitism               =  2 
Crossover probability =  0.5 
Mutation probability  =  0.01 

GA results: 
Iterations             = 100 
Fitness function value = 0.9891231 
Solution = 
     Max_termperature Min_temperature Dewpoint Precipitation Sea_level_pressure
[1,]                1               1        1             0                  1
     Standard_pressure Visibility Wind_speed Max_wind_speed
[1,]                 1          1          1              0

In [19]:
tmp_x <- paste(colnames(train)[best_var_idx], collapse=" + ")
tmp_xy <- paste("train$Mean_temperature ~ ", tmp_x, collapse = "") 
perf_mat[6,1] <- tmp_xy
as.formula(tmp_xy) 

gn_model <- lm(tmp_xy, data = train)
summary(gn_model)
perf_mat[6,2] <- summary(gn_model)$adj.r.squared

pred <- predict(gn_model, newdata = val)
perf_mat[6,4:6] <- perf_eval_reg(val$Mean_temperature, pred)
perf_mat

train$Mean_temperature ~ Max_termperature + Min_temperature + 
    Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + 
    Wind_speed


Call:
lm(formula = tmp_xy, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.1462 -1.0348 -0.0791  0.9466  5.1278 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        48.88231    0.09963 490.656   <2e-16 ***
Max_termperature    9.55883    0.26825  35.634   <2e-16 ***
Min_temperature     4.70258    0.32695  14.383   <2e-16 ***
Dewpoint            0.36353    0.26767   1.358   0.1757    
Sea_level_pressure -1.27200    0.72929  -1.744   0.0824 .  
Standard_pressure   0.69267    0.57471   1.205   0.2293    
Visibility          0.28994    0.13069   2.219   0.0274 *  
Wind_speed          0.19787    0.12422   1.593   0.1125    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.569 on 242 degrees of freedom
Multiple R-squared:  0.9894,	Adjusted R-squared:  0.9891 
F-statistic:  3236 on 7 and 242 DF,  p-value: < 2.2e-16


,input_formula,Adj.R^2,Time,RMSE,MAE,MAPE
full model,Max_termperature+Min_temperature+Dewpoint+Precipitation+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed+Max_wind_speed+Mean_temperature,0.989044028361043,0,0.184118990750289,1.28235010615204,2.83162660862481
Exhaustive Search,Max_termperature+Min_temperature+Dewpoint+Sea_level_pressure+Standard_pressure+Visibility+Wind_speed,0.989123124838608,0.650000000000002,0.177143175569796,1.2808548015898,2.82939076638354
Forward,train$Mean_temperature ~ Max_termperature + Min_temperature + Sea_level_pressure + Visibility + Standard_pressure,0.989049139340133,0.0300000000000011,0.16406155216688,1.31114759735903,2.89613751259997
Backward,train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Visibility + Wind_speed,0.989102865435691,0.0299999999999976,0.177493145133962,1.29580955253699,2.85758992183211
Stepwise,train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Visibility + Wind_speed,0.989102865435691,0.0300000000000011,0.177493145133962,1.29580955253699,2.85758992183211
Genetic,train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,4.01,0.177143175569796,1.2808548015898,2.82939076638354


# [Q5] Genetic Algorithm에서 변경 가능한 하이퍼 파라미터들(population size, Cross-over rate, Mutation rate 등)에 대해 최소한 세 가지 이상의 후보 값들을 선정하고 각 조합에 대한 변수 선택 결과를 비교해 보시오. 최종 결과에 가장 큰 영향을 미치는 하이퍼파라미터는 무엇으로 나타났는가? 왜 그런 결과가 나타났다고 생각하는가? 

In [20]:
# 파라미터 실험 결과들을 저장할 매트릭스 생성
params_mat <- matrix(0, nrow = 10, ncol = 6)
rownames(params_mat) <- c("popSize(10)","popSize(150)","pcrossover(0.2)","pcrossover(0.8)","pmutation(0.05)","pmutation(0.15)","maxiter(50)","maxiter(150)","elitism(0)","elitism(4)")
colnames(params_mat) <- c("input_formula","Adj.R^2","Time","RMSE", "MAE", "MAPE")

# for문을 돌리기 위해 10가지 경우의 파라미터들은 모두 저장
popSize <- c(10,150,50,50,50,50,50,50,50,50)
pcrossover <- c(0.5,0.5,0.2,0.8,0.5,0.5,0.5,0.5,0.5,0.5)
pmutation <- c(0.1,0.1,0.1,0.1,0.05,0.15,0.1,0.1,0.1,0.1)
maxiter <- c(100,100,100,100,100,100,50,150,100,100)
elitism <- c(2,2,2,2,2,2,2,2,0,4)

In [21]:
for(i in 1:10){
    start_time <- proc.time()
    GN <- ga(type = "binary", fitness = fitness, nBits = ncol(x_train), 
            names = colnames(x_train), popSize = popSize[i], pcrossover = pcrossover[i], 
            pmutation = pmutation[i], maxiter = maxiter[i], elitism = elitism[i], seed = 41)
    end_time <- proc.time()
    params_mat[i,3] <- (end_time - start_time)[3]
    best_var_idx <- which(GN@solution == 1)
    colnames(train)[best_var_idx]
    
    tmp_x <- paste(colnames(train)[best_var_idx], collapse=" + ")
    tmp_xy <- paste("train$Mean_temperature ~ ", tmp_x, collapse = "") 
    params_mat[i,1] <- tmp_xy
    as.formula(tmp_xy) 

    gn_model <- lm(tmp_xy, data = train)
    summary(gn_model)
    params_mat[i,2] <- summary(gn_model)$adj.r.squared

    pred <- predict(gn_model, newdata = val)
    params_mat[i,4:6] <- perf_eval_reg(val$Mean_temperature, pred)
  
}
params_mat

,input_formula,Adj.R^2,Time,RMSE,MAE,MAPE
popSize(10),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,0.82,0.177143175569796,1.2808548015898,2.82939076638354
popSize(150),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,12.51,0.177143175569796,1.2808548015898,2.82939076638354
pcrossover(0.2),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,1.91,0.177143175569796,1.2808548015898,2.82939076638354
pcrossover(0.8),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,6.75,0.177143175569796,1.2808548015898,2.82939076638354
pmutation(0.05),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,4.08,0.177143175569796,1.2808548015898,2.82939076638354
pmutation(0.15),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,4.34,0.177143175569796,1.2808548015898,2.82939076638354
maxiter(50),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,2.16,0.177143175569796,1.2808548015898,2.82939076638354
maxiter(150),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,7.2,0.177143175569796,1.2808548015898,2.82939076638354
elitism(0),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Visibility + Wind_speed + Max_wind_speed,0.989068886238476,3.83000000000001,0.184467351349441,1.29575934398128,2.85901741237431
elitism(4),train$Mean_temperature ~ Max_termperature + Min_temperature + Dewpoint + Sea_level_pressure + Standard_pressure + Visibility + Wind_speed,0.989123124838608,3.94,0.177143175569796,1.2808548015898,2.82939076638354


In [23]:
# 각 파라미터 당 3개씩 243개의 조합을 만들어서 포문으로 돌림

# 파라미터 실험 결과들을 저장할 매트릭스 생성
params_mat <- matrix(0, nrow = 243, ncol = 10)
colnames(params_mat) <- c("Popsize","pcrossover","pmutation","maxiter","elitism","input_formula","Adj.R^2","RMSE", "MAE", "MAPE")

#파라미터 조합
popSize <- c(10,50,150)
pcrossover <- c(0.2,0.5,0.8)
pmutation <- c(0.05,0.1,0.15)
maxiter <- c(50,100,150)
elitism <- c(0,2,4)
count <- 0

#포문 돌리는 부분
for (i in 1:3){
    for(j in 1:3){
        for(k in 1:3){
            for(l in 1:3){
                for(m in 1:3){
                    count <- count +1
                    GN <- ga(type = "binary", fitness = fitness, nBits = ncol(x_train), 
                            names = colnames(x_train), popSize = popSize[i], pcrossover = pcrossover[j], 
                            pmutation = pmutation[k], maxiter = maxiter[l], elitism = elitism[m], seed = 41)
                    params_mat[count,1] <- popSize[i]
                    params_mat[count,2] <- pcrossover[j]
                    params_mat[count,3] <- pmutation[k]
                    params_mat[count,4] <- maxiter[l]
                    params_mat[count,5] <- elitism[m]

                    best_var_idx <- which(GN@solution == 1)
                    colnames(train)[best_var_idx]
    
                    tmp_x <- paste(colnames(train)[best_var_idx], collapse=" + ")
                    tmp_xy <- paste("train$Mean_temperature ~ ", tmp_x, collapse = "") 
                    params_mat[count,6] <- tmp_xy
                    as.formula(tmp_xy) 

                    gn_model <- lm(tmp_xy, data = train)
                    summary(gn_model)
                    params_mat[count,7] <- summary(gn_model)$adj.r.squared

                    pred <- predict(gn_model, newdata = val)
                    params_mat[count,8:10] <- perf_eval_reg(val$Mean_temperature, pred)
                }
            }
        }
    }
}

In [88]:
# 매트릭스를 데이터프레임으로 변환
params_df <- as.data.frame(params_mat)
params_df <- params_df[,-6]
# factor변수를 실수 변수로 변환
params_df[,6] <- as.numeric(as.character(unlist(params_df[,6])))
params_df[,7] <- as.numeric(as.character(unlist(params_df[,7])))
params_df[,8] <- as.numeric(as.character(unlist(params_df[,8])))
params_df[,9] <- as.numeric(as.character(unlist(params_df[,9])))

In [89]:
head(params_df)

,Popsize,pcrossover,pmutation,maxiter,elitism,Adj.R^2,RMSE,MAE,MAPE
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,0.2,0.05,50,0,0.9889963,0.1936440,1.310041,2.881597
2,10,0.2,0.05,50,2,0.9891231,0.1771432,1.280855,2.829391
3,10,0.2,0.05,50,4,0.9891231,0.1771432,1.280855,2.829391
4,10,0.2,0.05,100,0,0.9890317,0.1870142,1.308264,2.875367
5,10,0.2,0.05,100,2,0.9891231,0.1771432,1.280855,2.829391
6,10,0.2,0.05,100,4,0.9891231,0.1771432,1.280855,2.829391


In [90]:
# popsize칼럼을 가지고 gropby하여 평가지표들의 평균을 구한다.
aggregate(params_df[,6:9],list(params_df[,1]),mean)

Group.1,Adj.R^2,RMSE,MAE,MAPE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
10,0.9836315,0.1999091,1.455183,3.189525
150,0.9891017,0.1752145,1.283083,2.833142
50,0.9888567,0.1765118,1.292920,2.852346


In [84]:
# pcrossover칼럼을 가지고 gropby하여 평가지표들의 평균을 구한다.
aggregate(params_df[,6:9],list(params_df[,2]),mean)

Group.1,Adj.R^2,RMSE,MAE,MAPE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
0.2,0.9878037,0.1841486,1.325460,2.916943
0.5,0.9882671,0.1779740,1.316963,2.905130
0.8,0.9855191,0.1895128,1.388762,3.052941


In [91]:
# pmutation칼럼을 가지고 gropby하여 평가지표들의 평균을 구한다.
aggregate(params_df[,cols],list(params_df[,3]),mean)

Group.1,Adj.R^2,RMSE,MAE,MAPE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
0.05,0.9862195,0.1943891,1.382394,3.031694
0.1,0.9866259,0.1848280,1.351271,2.973605
0.15,0.9887445,0.1724184,1.297520,2.869714


In [92]:
# maxiter칼럼을 가지고 gropby하여 평가지표들의 평균을 구한다.
aggregate(params_df[,6:9],list(params_df[,4]),mean)

Group.1,Adj.R^2,RMSE,MAE,MAPE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
100,0.9878148,0.1798118,1.325500,2.923043
150,0.9866662,0.1845464,1.354888,2.982530
50,0.9871089,0.1872772,1.350797,2.969441


In [93]:
# elitism칼럼을 가지고 gropby하여 평가지표들의 평균을 구한다.
aggregate(params_df[,6:9],list(params_df[,5]),mean)

Group.1,Adj.R^2,RMSE,MAE,MAPE
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.9833439,0.1973447,1.469291,3.215884
2,0.9891229,0.1771475,1.281039,2.829739
4,0.9891231,0.1771432,1.280855,2.829391
